In [1]:
import os
from ROOT import *
from ML_Clasification import LQ_Log_Reg

Welcome to JupyROOT 6.22/06


2022-09-28 09:13:01.143480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /Collider/ROOT/installROOT/lib::/Collider/ROOT/installROOT/lib/:/Collider/fastjet/lib:/Collider/fastjet/lib64:/Collider/LHAPDF/lib
2022-09-28 09:13:01.143517: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
folder_out=os.path.join(
    os.path.dirname(os.getcwd()),
    "03_delphes_analysis"
)
csv_files_path=os.path.join(folder_out,"Selected_Events")
masses=[250,500,750,1000,1250,1500,1750,2000]
gErrorIgnoreLevel=kError+1

In [3]:
channel="b_tau_tau_hadronic"
for mass in masses:
    modelo=LQ_Log_Reg(mass,[channel],csv_files_path)
    images_folder=os.path.join(f"MLQ_{mass}",f"{channel}")
    os.system(f"rm -rf '{images_folder}'")
    os.system(f"mkdir -p '{images_folder}'")
    histograms=modelo.get_discriminator_histograms(images_folder)
print(csv_files_path)

/disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/03_delphes_analysis/Selected_Events


Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_tau_tau_hadronic/LQ_LQ_0250.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_tau_tau_hadronic/Tau_LQ_0250.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_tau_tau_hadronic/ttbar.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_tau_tau_hadronic/stop.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_tau_tau_hadronic/z_jets.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_tau_tau_hadronic/w_jets.png has been create

In [4]:
channel="b_b_tau_tau_hadronic"
for mass in masses:
    modelo=LQ_Log_Reg(mass,[channel],csv_files_path)
    images_folder=os.path.join(f"MLQ_{mass}",f"{channel}")
    os.system(f"rm -rf '{images_folder}'")
    os.system(f"mkdir -p '{images_folder}'")
    histograms=modelo.get_discriminator_histograms(images_folder)
print(csv_files_path)

/disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/03_delphes_analysis/Selected_Events


Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_b_tau_tau_hadronic/LQ_LQ_0250.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_b_tau_tau_hadronic/Tau_LQ_0250.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_b_tau_tau_hadronic/ttbar.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_b_tau_tau_hadronic/stop.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_b_tau_tau_hadronic/z_jets.png has been created
Info in <TCanvas::Print>: png file /disco4/SIMULACIONES/Cristian/Pheno_BSM/Leptoquarks_searches/04_ML_analysis/MLQ_250/b_b_tau_tau_hadronic/w_jets.png has

In [ ]:
channel="b_tau_tau_semileptonic"
for mass in masses:
    modelo=LQ_Log_Reg(mass,[channel],csv_files_path)
    images_folder=os.path.join(f"MLQ_{mass}",f"{channel}")
    os.system(f"rm -rf '{images_folder}'")
    os.system(f"mkdir -p '{images_folder}'")
    histograms=modelo.get_discriminator_histograms(images_folder)
print(csv_files_path)

In [ ]:
channel="b_b_tau_tau_semileptonic"
for mass in masses:
    modelo=LQ_Log_Reg(mass,[channel],csv_files_path)
    images_folder=os.path.join(f"MLQ_{mass}",f"{channel}")
    os.system(f"rm -rf '{images_folder}'")
    os.system(f"mkdir -p '{images_folder}'")
    histograms=modelo.get_discriminator_histograms(images_folder)
print(csv_files_path)

x=[]
y=[]
n=100
for i in range(n+1):
    x+=[i*(1./float(n))]
    y+=[get_sig_by_threshold(histograms,x[i])]
x  = np.array(  x  )
y  = np.array(  y  )
print(y)